In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models

data = pd.read_csv('blackhole.csv')
print(f'Dataset shape: {data.shape}')
data.head()

Dataset shape: (404134, 18)


,time,source,destination,length,info,transmission_rate_per_1000_ms,reception_rate_per_1000_ms,transmission_average_per_sec,reception_average_per_sec,transmission_count_per_sec,reception_count_per_sec,transmission_total_duration_per_sec,reception_total_duration_per_sec,dao,dis,dio,category,label
0,0.037,39,9999,0.0,1.0,0.000000,0.671176,0.000000,0.499879,0.000000,0.671176,0.539313,0.570032,0.0,0.0,0.000000,Normal,0
1,0.037,39,9999,0.0,1.0,0.000000,0.649873,0.000000,0.505234,0.000000,0.649873,0.264704,0.530547,0.0,0.0,0.000000,Normal,0
2,0.038,39,9999,0.0,1.0,0.671176,0.652361,0.462516,0.501327,0.671768,0.652361,0.546376,1.000000,0.0,0.0,0.690115,Blackhole,1
3,0.045,39,9999,0.0,1.0,0.000000,0.633786,0.000000,0.517346,0.000000,0.634105,0.585425,0.553276,0.0,0.0,0.000000,Normal,0
4,0.046,39,9999,0.0,1.0,0.000000,0.630378,0.000000,0.538789,0.000000,0.630378,0.443171,0.615377,0.0,0.0,0.000000,Normal,0


In [2]:
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

# Upsample minority class (blackhole)
blackhole_upsampled = resample(blackhole, 
                               replace=True,    # sample with replacement
                               n_samples=len(normal), # match number in majority class
                               random_state=42) # reproducible results

# Combine majority and upsampled minority
# Identify and inspect categorical columns
print(blackhole_upsampled.dtypes)

# If there are any categorical columns, convert them to numerical using one-hot encoding
upsampled_encoded = pd.get_dummies(blackhole_upsampled, drop_first=True)

# Preprocessing: feature scaling after encoding
X = upsampled_encoded.drop('label', axis=1)
y = upsampled_encoded['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


time                                   float64
source                                   int64
destination                              int64
length                                 float64
info                                   float64
transmission_rate_per_1000_ms          float64
reception_rate_per_1000_ms             float64
transmission_average_per_sec           float64
reception_average_per_sec              float64
transmission_count_per_sec             float64
reception_count_per_sec                float64
transmission_total_duration_per_sec    float64
reception_total_duration_per_sec       float64
dao                                    float64
dis                                    float64
dio                                    float64
category                                object
label                                    int64
dtype: object


In [3]:
class MAMLModel(tf.keras.Model):
    def __init__(self):
        super(MAMLModel, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu')
        self.fc2 = layers.Dense(32, activation='relu')
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        return self.out(x)

# Model training setup
def maml_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X)
        predictions = tf.squeeze(predictions)  # Ensure predictions have the same shape as y
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


# Federated Learning setup
def federated_train(X_splits, y_splits, num_rounds=10):
    global_model = MAMLModel()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            maml_train_step(global_model, optimizer, X, y)

    return global_model

# Simulate distributed datasets for federated learning
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Federated training
global_model = federated_train(X_splits, y_splits)



C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10


In [4]:
y_pred = global_model(X_test)
y_pred = np.round(y_pred)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))

Confusion Matrix:
[[53971]]

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     53971

    accuracy                           1.00     53971
   macro avg       1.00      1.00      1.00     53971
weighted avg       1.00      1.00      1.00     53971


Accuracy Score:
1.0


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [5]:
# Upsample minority class
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

blackhole_upsampled = resample(blackhole, 
                               replace=True,    
                               n_samples=len(normal),
                               random_state=42)

# Combine and encode
upsampled_encoded = pd.get_dummies(pd.concat([normal, blackhole_upsampled]), drop_first=True)

# Preprocessing
X = upsampled_encoded.drop('label', axis=1)
y = upsampled_encoded['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
# Define MAML model with Dropout
class MAMLModel(tf.keras.Model):
    def __init__(self):
        super(MAMLModel, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu')
        self.dropout1 = layers.Dropout(0.5)  # Dropout layer
        self.fc2 = layers.Dense(32, activation='relu')
        self.dropout2 = layers.Dropout(0.5)  # Dropout layer
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.dropout1(x)  # Apply dropout
        x = self.fc2(x)
        x = self.dropout2(x)  # Apply dropout
        return self.out(x)

# Model training setup
def maml_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X)
        predictions = tf.squeeze(predictions)
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Learning setup
def federated_train(X_splits, y_splits, num_rounds=10):
    global_model = MAMLModel()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            maml_train_step(global_model, optimizer, X, y)

    return global_model

# Simulate distributed datasets for federated learning
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Federated training
global_model = federated_train(X_splits, y_splits)

# Evaluate the model
y_pred = global_model(X_test)
y_pred = np.round(y_pred)

C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10


In [6]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))

Confusion Matrix:


ValueError: Found input variables with inconsistent numbers of samples: [107941, 53971]

In [11]:
import pandas as pd
import numpy as np

# Function to get user input and check the model
def get_user_input_and_predict(model, scaler):
    # Define feature names
    feature_names = [
        'time', 'source', 'destination', 'length', 'info',
        'transmission_rate_per_1000_ms', 'reception_rate_per_1000_ms',
        'transmission_average_per_sec', 'reception_average_per_sec',
        'transmission_count_per_sec', 'reception_count_per_sec',
        'transmission_total_duration_per_sec', 'reception_total_duration_per_sec',
        'dao', 'dis', 'dio', 'category'
    ]

    # Get user input
    user_input = {}
    for feature in feature_names:
        user_input[feature] = input(f"Enter value for {feature}: ")

    # Create a DataFrame from user input
    input_data = pd.DataFrame(user_input, index=[0])

    # Call the check_model function to make predictions
    predicted_labels = check_model(model, input_data, scaler)
    
    print("Predicted Label:", predicted_labels[0])

# Check model function (as defined earlier)
def check_model(model, input_data, scaler):
    # Convert categorical variables to dummy variables
    input_data_encoded = pd.get_dummies(input_data, drop_first=True)

    # Ensure the same features are present
    missing_cols = set(X.columns) - set(input_data_encoded.columns)
    for col in missing_cols:
        input_data_encoded[col] = 0  # Add missing columns with 0s

    # Reorder columns to match training data
    input_data_encoded = input_data_encoded[X.columns]

    # Scale the features
    input_data_scaled = scaler.transform(input_data_encoded)

    # Make predictions
    predictions = model.predict(input_data_scaled)
    predictions = np.round(predictions).astype(int)

    return predictions

# Example usage
# Assuming global_model and scaler are available
get_user_input_and_predict(global_model, scaler)

KeyboardInterrupt: Interrupted by user

In [18]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define preprocessing function
def preprocess_data(X, y, test_size=0.2, random_state=42):
    # Convert to NumPy arrays if not already
    if not isinstance(X, np.ndarray):
        X = np.array(X)
    if not isinstance(y, np.ndarray):
        y = np.array(y)
    
    # Scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test


# Updated maml_train_step function
def maml_train_step(model, optimizer, X, y):
    X, y = preprocess_data(X, y)  # Preprocess data
    with tf.GradientTape() as tape:
        predictions = model(X)
        predictions = tf.squeeze(predictions)  # Ensure correct shape
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Training Setup with Adjustments
def federated_train(X_splits, y_splits, num_rounds=20, dropout_rate=0.2, learning_rate=0.0005):
    global_model = AdvancedMAMLModel(dropout_rate=dropout_rate)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            maml_train_step(global_model, optimizer, X, y)
    return global_model

# Preprocess and split the dataset
X_train, X_test, y_train, y_test = preprocess_data(X, y)  # Provide full dataset X, y
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Perform federated training
global_model = federated_train(X_splits, y_splits)

# Evaluate the model
y_pred = global_model(X_test).numpy()
y_pred = (y_pred > 0.5).astype(int)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))



Round 1


ValueError: too many values to unpack (expected 2)

In [9]:
# Updating the MAML and Federated Learning Model script with fixes and improvements.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models

# Define updated MAML model with Dropout and Batch Normalization
class AdvancedMAMLModel(tf.keras.Model):
    def __init__(self, dropout_rate=0.5):
        super(AdvancedMAMLModel, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(dropout_rate)
        self.fc2 = layers.Dense(32, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.dropout2 = layers.Dropout(dropout_rate)
        self.out = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.dropout2(x)
        return self.out(x)

# Preprocessing function to scale data
def preprocess_data(X, y, scaler=None, fit_scaler=True):
    if not isinstance(X, np.ndarray):
        X = np.array(X)
    if not isinstance(y, np.ndarray):
        y = np.array(y)
    
    if fit_scaler and scaler is None:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = scaler.transform(X)
    
    return X_scaled, y, scaler

# Federated training setup
def maml_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        predictions = tf.squeeze(predictions)
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss.numpy()

def federated_train(X_splits, y_splits, num_rounds=10, dropout_rate=0.5, learning_rate=0.001):
    global_model = AdvancedMAMLModel(dropout_rate=dropout_rate)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}/{num_rounds}")
        for X, y in zip(X_splits, y_splits):
            maml_train_step(global_model, optimizer, tf.convert_to_tensor(X, dtype=tf.float32), tf.convert_to_tensor(y, dtype=tf.float32))
    
    return global_model

# Example data preprocessing and training pipeline
def main():
    # Example: Load and preprocess data (replace with actual dataset path)
    data = pd.read_csv('blackhole.csv')
    normal = data[data['label'] == 0]
    blackhole = data[data['label'] == 1]
    
    # Upsample minority class
    blackhole_upsampled = resample(blackhole, replace=True, n_samples=len(normal), random_state=42)
    combined_data = pd.concat([normal, blackhole_upsampled])
    
    # Encode categorical features
    encoded_data = pd.get_dummies(combined_data, drop_first=True)
    X = encoded_data.drop('label', axis=1)
    y = encoded_data['label']
    
    # Preprocess and split data
    X_scaled, y, scaler = preprocess_data(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Simulate federated learning data
    X_splits = np.array_split(X_train, 5)
    y_splits = np.array_split(y_train, 5)
    
    # Train the global model
    global_model = federated_train(X_splits, y_splits, num_rounds=10)
    
    # Evaluate the model
    y_pred = global_model(tf.convert_to_tensor(X_test, dtype=tf.float32)).numpy()
    y_pred = (y_pred > 0.5).astype(int)
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nAccuracy Score:")
    print(accuracy_score(y_test, y_pred))

# Execute the pipeline
# Uncomment the following line if running in an interactive environment
# main()


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Load dataset
data = pd.read_csv('blackhole.csv')
print(f'Dataset shape: {data.shape}')

# Data preprocessing: Upsampling minority class
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

# Upsample the minority class
blackhole_upsampled = resample(
    blackhole, replace=True, n_samples=len(normal), random_state=42
)
balanced_data = pd.concat([normal, blackhole_upsampled])

# One-hot encode categorical columns if any
balanced_data_encoded = pd.get_dummies(balanced_data, drop_first=True)

# Separate features and labels
X = balanced_data_encoded.drop('label', axis=1)
y = balanced_data_encoded['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the enhanced MAML model
class EnhancedMAMLModel(tf.keras.Model):
    def __init__(self):
        super(EnhancedMAMLModel, self).__init__()
        self.fc1 = layers.Dense(128, activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(0.3)
        self.fc2 = layers.Dense(64, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.dropout2 = layers.Dropout(0.3)
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.dropout2(x)
        return self.out(x)

# Model training setup
def maml_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        predictions = tf.squeeze(predictions)  # Ensure predictions have the correct shape
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Learning setup
def federated_train(X_splits, y_splits, num_rounds=10):
    global_model = EnhancedMAMLModel()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            maml_train_step(global_model, optimizer, X, y)

    return global_model

# Split data into 5 subsets for federated learning
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Introduce EarlyStopping and Learning Rate Scheduler
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Federated training
global_model = federated_train(X_splits, y_splits)

# Evaluate the model
y_pred = global_model(X_test).numpy()
y_pred = np.round(y_pred)

# Output results
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))


Dataset shape: (404134, 18)


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Confusion Matrix:
[[53962     1]
 [    0 53978]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53963
           1       1.00      1.00      1.00     53978

    accuracy                           1.00    107941
   macro avg       1.00      1.00      1.00    107941
weighted avg       1.00      1.00      1.00    107941


Accuracy Score:
0.9999907356796769


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers, callbacks, models
from tensorflow.keras.optimizers import Adam

# Load dataset
data = pd.read_csv('blackhole.csv')
print(f'Dataset shape: {data.shape}')

# Data preprocessing
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

# Upsample the minority class
blackhole_upsampled = resample(
    blackhole, replace=True, n_samples=len(normal), random_state=42
)
balanced_data = pd.concat([normal, blackhole_upsampled])

# Encode categorical data and separate features and labels
balanced_data_encoded = pd.get_dummies(balanced_data, drop_first=True)
X = balanced_data_encoded.drop('label', axis=1)
y = balanced_data_encoded['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split with validation set
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define a more structured MAML model
class EnhancedMAMLModel(tf.keras.Model):
    def __init__(self, hidden_units_1=128, hidden_units_2=64, dropout_rate=0.3):
        super(EnhancedMAMLModel, self).__init__()
        self.fc1 = layers.Dense(hidden_units_1, activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(dropout_rate)
        self.fc2 = layers.Dense(hidden_units_2, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.dropout2 = layers.Dropout(dropout_rate)
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.fc1(inputs)
        x = self.bn1(x, training=training)
        x = self.dropout1(x, training=training)
        x = self.fc2(x)
        x = self.bn2(x, training=training)
        x = self.dropout2(x, training=training)
        return self.out(x)

# Model training setup
def maml_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        predictions = tf.squeeze(predictions)
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Learning setup with local training
def federated_train(X_splits, y_splits, X_val, y_val, num_rounds=10, local_epochs=2):
    global_model = EnhancedMAMLModel()
    global_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    _ = global_model(tf.keras.Input(shape=(X_splits[0].shape[1],)))  # Build global model

    # Validation tracking
    val_losses = []

    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        
        aggregated_weights = [np.zeros_like(w) for w in global_model.get_weights()]  # Initialize aggregated weights
        for X, y in zip(X_splits, y_splits):
            local_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
            local_model = EnhancedMAMLModel()
            _ = local_model(tf.keras.Input(shape=(X.shape[1],)))  # Build local model
            local_model.set_weights(global_model.get_weights())  # Sync with global model

            # Local training on each client
            for _ in range(local_epochs):
                maml_train_step(local_model, local_optimizer, X, y)

            # Accumulate weights for aggregation
            local_weights = local_model.get_weights()
            for i, (global_w, local_w) in enumerate(zip(aggregated_weights, local_weights)):
                aggregated_weights[i] += local_w / len(X_splits)

        # Update global model weights after aggregation
        global_model.set_weights(aggregated_weights)

        # Evaluate on the validation set
        val_predictions = global_model(X_val, training=False)
        y_val_reshaped = tf.reshape(y_val, (-1, 1))  # Ensure y_val matches val_predictions
        val_loss = tf.keras.losses.binary_crossentropy(y_val_reshaped, val_predictions).numpy().mean()

        val_losses.append(val_loss)
        print(f"Validation Loss: {val_loss:.4f}")

    return global_model


# Split training data for federated learning
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Train the model
global_model = federated_train(X_splits, y_splits, X_val, y_val)

# Evaluate the model on the test set
y_pred = global_model(X_test, training=False).numpy()
y_pred = np.round(y_pred)

# Output results
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))


Dataset shape: (404134, 18)


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)



Round 1
Validation Loss: 0.7123
Round 2
Validation Loss: 0.6673
Round 3
Validation Loss: 0.6238
Round 4
Validation Loss: 0.5824
Round 5
Validation Loss: 0.5417
Round 6
Validation Loss: 0.5012
Round 7
Validation Loss: 0.4612
Round 8
Validation Loss: 0.4212
Round 9
Validation Loss: 0.3817
Round 10
Validation Loss: 0.3432
Confusion Matrix:
[[39633   750]
 [    0 40573]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     40383
           1       0.98      1.00      0.99     40573

    accuracy                           0.99     80956
   macro avg       0.99      0.99      0.99     80956
weighted avg       0.99      0.99      0.99     80956


Accuracy Score: 0.9907357082859826
Precision: 0.9818503012849986
Recall: 1.0
F1-Score: 0.9908420435674514


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers, callbacks, models
from tensorflow.keras.optimizers import Adam

# Load dataset
data = pd.read_csv('blackhole.csv')
print(f'Dataset shape: {data.shape}')

# Data preprocessing
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

# Upsample the minority class
blackhole_upsampled = resample(
    blackhole, replace=True, n_samples=len(normal), random_state=42
)
balanced_data = pd.concat([normal, blackhole_upsampled])

# Encode categorical data and separate features and labels
balanced_data_encoded = pd.get_dummies(balanced_data, drop_first=True)
X = balanced_data_encoded.drop('label', axis=1)
y = balanced_data_encoded['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split with validation set
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define Enhanced MAML Model with Regularization and Dropout
class EnhancedMAMLModel(tf.keras.Model):
    def __init__(self):
        super(EnhancedMAMLModel, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dropout1 = layers.Dropout(0.4)
        self.fc2 = layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dropout2 = layers.Dropout(0.4)
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        return self.out(x)

# MAML Training Step
def maml_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        predictions = tf.squeeze(predictions)
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Training Setup
def federated_train(X_splits, y_splits, X_val, y_val, num_rounds=15, local_epochs=3):
    global_model = EnhancedMAMLModel()
    global_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0008)
    _ = global_model(tf.keras.Input(shape=(X_splits[0].shape[1],)))

    val_losses = []

    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        
        aggregated_weights = [np.zeros_like(w) for w in global_model.get_weights()]
        for X, y in zip(X_splits, y_splits):
            local_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
            local_model = EnhancedMAMLModel()
            _ = local_model(tf.keras.Input(shape=(X.shape[1],)))
            local_model.set_weights(global_model.get_weights())

            # Local training on each client
            for _ in range(local_epochs):
                maml_train_step(local_model, local_optimizer, X, y)

            # Accumulate weights for aggregation
            local_weights = local_model.get_weights()
            for i, (global_w, local_w) in enumerate(zip(aggregated_weights, local_weights)):
                aggregated_weights[i] += local_w / len(X_splits)

        # Update global model weights after aggregation
        global_model.set_weights(aggregated_weights)

        # Evaluate on the validation set
        val_predictions = global_model(X_val, training=False)
        y_val_reshaped = tf.reshape(y_val, (-1, 1))
        val_loss = tf.keras.losses.binary_crossentropy(y_val_reshaped, val_predictions).numpy().mean()

        val_losses.append(val_loss)
        print(f"Validation Loss: {val_loss:.4f}")

    return global_model

# Split training data for federated learning
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Train the model
global_model = federated_train(X_splits, y_splits, X_val, y_val)

# Evaluate the model on the test set
y_pred = global_model(X_test, training=False).numpy()
y_pred = np.round(y_pred)

# Output results
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))


Dataset shape: (404134, 18)


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Round 1
Validation Loss: 0.7882
Round 2
Validation Loss: 0.7605
Round 3
Validation Loss: 0.7346
Round 4
Validation Loss: 0.7103
Round 5
Validation Loss: 0.6873
Round 6
Validation Loss: 0.6657
Round 7
Validation Loss: 0.6450
Round 8
Validation Loss: 0.6251
Round 9
Validation Loss: 0.6056
Round 10
Validation Loss: 0.5866
Round 11
Validation Loss: 0.5678
Round 12
Validation Loss: 0.5492
Round 13
Validation Loss: 0.5307
Round 14
Validation Loss: 0.5122
Round 15
Validation Loss: 0.4937
Confusion Matrix:
[[39447   936]
 [ 4183 36390]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     40383
           1       0.97      0.90      0.93     40573

    accuracy                           0.94     80956
   macro avg       0.94      0.94      0.94     80956
weighted avg       0.94      0.94      0.94     80956


Accuracy Score: 0.9367681209545926
Precision: 0.9749236457161228
Recall: 0.8969018805609642
F1-Score: 0.934286704

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import librosa

# Load your dataset (assuming text, audio features, and labels)
# Sample dataset: 'text', 'audio_file_path', 'label'
data = pd.read_csv("mental_health_data.csv")

# Function to extract features from audio files
def extract_audio_features(file_name):
    y, sr = librosa.load(file_name, duration=30)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    return mfcc

# Extract audio features for all audio files
data['audio_features'] = data['audio_file_path'].apply(extract_audio_features)

# Text processing (TF-IDF vectorization)
vectorizer = TfidfVectorizer(max_features=3000)
X_text = vectorizer.fit_transform(data['text']).toarray()

# Combine text and audio features
X_audio = np.vstack(data['audio_features'].values)
X_combined = np.hstack((X_text, X_audio))

# Labels (assuming binary classification: 0 = no depression/suicidal tendency, 1 = depression/suicidal tendency)
y = data['label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Standardize the audio features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Multinomial Naive Bayes": MultinomialNB()
}

# Training and evaluation
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"\nModel: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

ModuleNotFoundError: No module named 'librosa'